In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras.layers import Dense, LeakyReLU, Dropout, Input, BatchNormalization
from keras.layers import Reshape, Conv2D, Conv2DTranspose, Flatten, Activation
from keras.models import Model,Sequential

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 텐서 플로의 정보 출력 억제하기
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # GPU 장치 지정

# tf.debugging.set_log_device_placement(True)   # 이거 쓰지 마셈 ㅈㄴ 출력 더러움

if not tf.config.list_physical_devices('GPU'):
    print("감지된 GPU가 없습니다. GPU가 없으면 LSTM과 CNN이 매우 느릴 수 있습니다.")

## 11.
_연습문제: 이미지 데이터셋을 처리하는 DCGAN을 훈련하고 이를 사용해 이미지를 생성해보세요. 경험 재생을 추가하고 도움이 되는지 확인하세요. 생성된 클래스를 제어할 수 있는 조건 GAN으로 바꾸어 시도해보세요._

### Load MNIST Dataset

* 'generator'의 'tanh' Activation 출력에 적합하도록 정규화

In [ ]:
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Noramlization
X_train = X_train.astype(np.float32) / 127.5 - 1

# Reshape
X_train = X_train.reshape(-1, 28, 28, 1)

### 'generator' Model
- 랜덤 벡터(잠재공간의 랜덤 포인트)를 입력받아 이미지 생성 
    - NOISE_DIM : 입력 랜덤 벡터 크기
- 'discriminator'를 속이도록 학습
    - 'Real Image'와 같은 'Fake Image' 생성이 목적

In [ ]:
'''NOISE_DIM = 10

generator = Sequential(name = 'generator')

generator.add(Dense(256 * 7 * 7, input_shape = (NOISE_DIM,)))
generator.add(LeakyReLU())
# 12544
generator.add(Reshape((7, 7, 256)))
# (14, 14, 128) 25088
generator.add(Conv2DTranspose(128, kernel_size = 3,
                              strides = 2,
                              padding = 'same'))
generator.add(BatchNormalization())
generator.add(LeakyReLU())
# (28, 28, 64) 50176
generator.add(Conv2DTranspose(64, kernel_size = 3,
                              strides = 2,
                              padding = 'same'))
generator.add(BatchNormalization())
generator.add(LeakyReLU())
# (28, 28, 1)
generator.add(Conv2D(1, kernel_size = 3, padding='same'))
generator.add(Activation('tanh'))
'''

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)
NOISE_DIM = 10

generator = keras.models.Sequential([
    keras.layers.Dense(256 * 7 * 7, input_shape=[NOISE_DIM]),
    keras.layers.Reshape(7, 7, 256),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(128, kernel_size=3, strides=2,padding ='SAME',
                                 activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(64, kernel_size=3, strides=2, padding='SAME', 
                                 activation="tanh")],
    name='generator')

In [ ]:
generator.summary()

### 'discriminator' Model

- 이미지를 입력받아 'Real Image'인지 'generator'가 생성한 'Fake Image'인지 판별
    - 이진 분류

In [ ]:
'''# from tensorflow.keras.initializers import RandomNormal

discriminator = Sequential(name = 'discriminator')

discriminator.add(Conv2D(32, kernel_size = 3,
                         strides = 2,
                         padding = 'same',
                         input_shape = (28, 28, 1)))
discriminator.add(LeakyReLU())
discriminator.add(Dropout(0.5))

discriminator.add(Conv2D(64, kernel_size = 3,strides = 2,
                         padding = 'same'))
discriminator.add(LeakyReLU())

discriminator.add(Conv2D(128, kernel_size=3,
                         strides=2,
                         padding='same'))
discriminator.add(LeakyReLU())
discriminator.add(Dropout(0.5))
discriminator.add(Flatten() )
discriminator.add(Dense(1, activation = 'sigmoid'))'''

In [ ]:
discriminator = keras.models.Sequential([

    keras.layers.Conv2D(32, kernel_size=3, strides=2, padding='SAME', 
                        activation=keras.layers.LeakyReLU(0.2),
                        input_shape=[28, 28, 1]),
    keras.layers.Dropout(0.5),
    
    keras.layers.Conv2D(64, kernel_size=3, strides=2, padding='SAME', 
                        activation=keras.layers.LeakyReLU(0.2)),
    
    keras.layers.Conv2D(128, kernel_size=3, strides=2, padding='SAME', 
                        activation=keras.layers.LeakyReLU(0.2)),
    keras.layers.Dropout(0.5),
    
    keras.layers.Flatten(),
    keras.layers.Dense(1, activation="sigmoid")], 
    name='discriminator')

In [ ]:
discriminator.summary()

#### 'discriminator' Compile
- 학습 설정

### 'adam' Optimizer
- beta_1 : 감쇠율 조정

In [ ]:
from keras.optimizers import Adam

adam = Adam(lr=0.0002, beta_1=0.5)

In [ ]:
discriminator.compile(loss='binary_crossentropy', optimizer=adam)

### 'GAN' Model

#### 'generator', 'discriminator' 연결
- 'gan' 모델에서 'generator'만 학습하도록 설정
    - disciriminator.trainable = False

In [ ]:
discriminator.trainable = False

# gan_input = Input(shape=(NOISE_DIM,))
# x = generator(gan_input) # generator
# output = discriminator(x) # discriminator

gan = keras.models.Sequential([generator, discriminator])

#### 'gan' Model

In [ ]:
# gan = Model(gan_input, output, name='gan')
gan.summary()

#### 'gan' Compile

In [ ]:
gan.compile(loss='binary_crossentropy', optimizer=adam)

### Define 'get_batches()' Function
- MNIST image batch 생성

In [ ]:
def get_batches(data, batch_size):
    batches = []

    for i in range(data.shape[0] // batch_size): # epoch
        batch = data[i * batch_size : (i + 1) * batch_size]
        batches.append(batch)
    
    return np.asarray(batches)

### 'visualize_training()' Function

In [ ]:
def visualize_training(epoch, d_losses, g_losses):
  
    # 오차 시각화
    # plt.figure(figsize=(8, 4))
    # plt.plot(d_losses, label='Discriminator Loss')
    # plt.plot(g_losses, label='Generatror Loss')
    # plt.xlabel('Epoch')
    # plt.ylabel('Loss')
    # plt.legend()
    # plt.show()
    # print('epoch: {}, Discriminator Loss: {}, Generator Loss: {}'.format(epoch, np.asarray(d_loss

    # 이미지 생성 결과 시각화
    print('epoch :', epoch)
    noise = np.random.normal(0, 1, size = (24, NOISE_DIM))
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(-1, 28, 28)
    
    plt.figure(figsize = (8, 4))
    
    for i in range(generated_images.shape[0]):
        plt.subplot(4, 6, i + 1)
        plt.imshow(generated_images[i], interpolation = 'nearest', cmap = 'Greys_r')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

### Define Loss

In [ ]:
# loss_function = keras.losses.BinaryCrossentropy()
# train_loss = keras.metrics.BinaryCrossentropy(name = 'train_loss')
# train_accuracy = keras.metrics.BinaryAccuracy(name = 'train_accuracy')

### Model Training

- 약 25분 

- .fit( )
    - 'epoch', 'batch_size' 지정 
- .train_on_batch( )
    - 전달 받은 모든 데이터를 사용하여 학습 진행
- 'generator'가 매번 새로운 'Fake Image'를 생성하여 '.train_on_batch( )' 사용

In [ ]:
%%time

EPOCHS = 50
BATCH_SIZE = 128

# 'discriminator', 'gan' Loss 저장 List 
d_losses = []
g_losses = []

for epoch in range(1, EPOCHS + 1): 
    # batch 별 학습
    for real_images in get_batches(X_train, BATCH_SIZE):
        # Random Noise 생성
        input_noise = np.random.uniform(-1, 1, size = [BATCH_SIZE, NOISE_DIM])

        # Fake Image 데이터 생성
        generated_images = generator.predict(input_noise)

        # 'gan' 학습용 X 데이터 정의
        x_dis = np.concatenate([real_images, generated_images])

        # 'gan' 학습용 y 데이터 정의
        y_dis = np.zeros(2 * BATCH_SIZE) 
        y_dis[:BATCH_SIZE] = 1
        
        # 'discriminator' 학습
        discriminator.trainable = True
        d_loss = discriminator.train_on_batch(x_dis, y_dis)

        # 'gan' 학습
        noise = np.random.uniform(-1, 1, size = [BATCH_SIZE, NOISE_DIM]) 
        y_gan = np.ones(BATCH_SIZE)

        # 'discriminator' 학습 정지 
        discriminator.trainable = False
        g_loss = gan.train_on_batch(noise, y_gan)
    
    d_losses.append(d_loss)
    g_losses.append(g_loss)

    # 생성 결과 시각화
    if epoch == 1 or epoch % 5 == 0:
        visualize_training(epoch, d_losses, g_losses)